In [1]:
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print(torch.cuda.is_available())

import detectron2
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
setup_logger()

import numpy as np
import os, json, cv2, random
import time

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import torchvision
import requests
print(torch.__version__, torch.cuda.is_available())

from detectron2.data.datasets import register_coco_instances
register_coco_instances("ball", {}, 'dataset/output_100.json', 'dataset')
register_coco_instances("bat", {}, 'dataset/output_100.json', 'dataset')

ball_metadata = MetadataCatalog.get("ball")
bat_metadata = MetadataCatalog.get("bat")

dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

from detectron2.engine import DefaultTrainer
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("ball", "bat")
cfg.DATASETS.TEST = ("ball", "bat")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")   # Let training initialize from model zoo
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # two class(ball, bat)

# ->> 처리
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_bat.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold for this model
cfg.DATASETS.TEST = ("ball", "bat")
predictor = DefaultPredictor(cfg)

#---- 선수 맞은편 카메라
cfg2 = get_cfg()
cfg2.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg2.DATASETS.TRAIN = ("ball",)
cfg2.DATASETS.TEST = ("ball", )
cfg2.DATALOADER.NUM_WORKERS = 2
cfg2.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")   # Let training initialize from model zoo
cfg2.MODEL.ROI_HEADS.NUM_CLASSES = 1 # one class(ball)

# ->> 처리
cfg2.MODEL.WEIGHTS = os.path.join(cfg2.OUTPUT_DIR, "model_final_ball.pth")
cfg2.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold for this model
cfg2.DATASETS.TEST = ("ball",)
predictor2 = DefaultPredictor(cfg2)
#----- 끝

# 함수 선언부
def lines_intersect(line1, line2):
    """Return True if line segment 'line1' and 'line2' intersect. False otherwise."""
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Compute vectors corresponding to lines
    px = x2 - x1
    py = y2 - y1
    qx = x4 - x3
    qy = y4 - y3

    # Cross product of vectors to determine if parallel
    cross_product = px * qy - qx * py
    if abs(cross_product) < 1e-8: # consider as zero
        return False  # parallel

    # Compute the intersection point
    dx = x1 - x3
    dy = y1 - y3
    t = (qx * dy - qy * dx) / cross_product
    u = (px * dy - py * dx) / cross_product

    return (0 <= t <= 1) and (0 <= u <= 1)

# 함수 선언부 끝

torch:  2.0 ; cuda:  2.0.0
detectron2: 0.2.1
True
2.0.0 True
WARNING [06/13 17:22:22 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/13 17:22:22 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json
WARNING [06/13 17:22:22 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/13 17:22:22 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json


Some model parameters or buffers are not found in the checkpoint:
proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [7]:
# 각 파일 path
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
 
# Create predictor and metadata objects
ball_metadata = MetadataCatalog.get("ball")
bat_metadata = MetadataCatalog.get("bat")
dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

predictor = DefaultPredictor(cfg)
predictor2 = DefaultPredictor(cfg2)

# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA) #벡엔드로 쿠다를 사용하여 속도향상을 꾀한다
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA) # 쿠다 디바이스에 계산 요청

inputWidth=320;
inputHeight=240;
inputScale=1.0/255;

start_time = time.time() # 시작 시간
ball_coordinates = []
points_buffer = {2: [], 8: [], 9: []} # 버퍼 초기화
drawn_line = None
line_timestamp = None  # 선의 타임스탬프 변수 초기화

# 선수 신체부위 추출용 끝

video_path1 = 'dataset/test15.mp4' # 측면카메라
cap1 = cv2.VideoCapture(video_path1)

video_path2 = 'dataset/test15.mp4' # 홈플레이트 카메라
cap2 = cv2.VideoCapture(video_path2)

while cv2.waitKey(1) <0:  #아무 키나 누르면 끝난다.
    #웹캠으로부터 영상 가져옴
    hasFrame, frame = cap1.read()  
    hasFrame2, frame2 = cap2.read() # 홈플레이트

    #영상이 커서 느리면 사이즈를 줄이자
    #frame=cv2.resize(frame,dsize=(1200,800),interpolation=cv2.INTER_AREA)
    
    #웹캠으로부터 영상을 가져올 수 없으면 웹캠 중지
    if not hasFrame or not hasFrame2:
        cv2.waitKey()
        break
    
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    inpBlob = cv2.dnn.blobFromImage(frame, inputScale, (inputWidth, inputHeight), (0, 0, 0), swapRB=False, crop=False)
    
    imgb=cv2.dnn.imagesFromBlob(inpBlob)
    #cv2.imshow("motion",(imgb[0]*255.0).astype(np.uint8))
    
    # network에 넣어주기
    net.setInput(inpBlob)

    # 결과 받아오기
    output = net.forward()
    
    outputs = predictor2(frame)
    v = Visualizer(frame[:, :, ::-1], metadata=ball_metadata)
    instances = outputs["instances"].to("cpu")
    confidence_threshold = 0.8 # Set your desired confidence threshold here

    for i in range(len(instances)):
        if instances.pred_classes[i] == 0: # 0번 클래스는 공입니다.
            confidence = instances.scores[i]
            if confidence > confidence_threshold:
                bbox = instances.pred_boxes[i]
                x1, y1, x2, y2 = bbox.tensor.numpy()[0]
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                radius = int((x2 - x1 + y2 - y1) / 4)
                cv2.circle(frame, (center_x, center_y), radius, (255, 0, 0), thickness=2)
                ball_coordinates.append((center_x, center_y))  # Add ball coordinates

    out = v.draw_instance_predictions(instances)

    # 키포인트 검출시 이미지에 그려줌
    points = []
    for i in [2,8,9]:
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]
    
        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (frameWidth * point[0]) / output.shape[3]
        y = (frameHeight * point[1]) / output.shape[2]

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
        if prob > 0.1 :   
            cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED) # circle(그릴곳, 원의 중심, 반지름, 색)
            cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
            points_buffer[i].append((x, y))  # 버퍼에 점 추가
        else :
            points.append(None)
            points_buffer[i] = []  # 버퍼 클리어

    # 각 점의 버퍼가 1초 이상의 데이터를 가지고 있다면
    if all([len(points_buffer[i]) >= 1*10 for i in [2, 8, 9]]):
        # 각 점의 평균 계산
        avg_points = {i: np.mean(points_buffer[i], axis=0) for i in [2, 8, 9]}
        shoulder_avg, waist_avg, knee_avg = avg_points[2], avg_points[8], avg_points[9]
        cal_avg = waist_avg[1] + ((shoulder_avg[1]-waist_avg[1])/2)

        # Draw the line
        drawn_line = (int(knee_avg[0]), int(knee_avg[1]), int(knee_avg[0]), int(cal_avg))
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)

    # 키 포인트 중 하나라도 감지되지 않으면, 최소 1초가 지났는지 확인
    if any(pt is None for pt in points):
        if line_timestamp is None or time.time() - line_timestamp >= 1:
            drawn_line = None
            line_timestamp = None

    # 이전에 선이 그려졌다면, 그 선을 그리고 타임스탬프를 업데이트
    if drawn_line is not None:
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)
        line_timestamp = time.time()
    
    cv2.imshow("Side Camera",frame)
# -----------홈플레이트 카메라 부분---------------
    outputs2 = predictor(frame2)
    v2 = Visualizer(frame2[:, :, ::-1], metadata=ball_metadata, scale=1)
    out2 = v2.draw_instance_predictions(outputs2["instances"].to("cpu"))
    output_frame2 = out2.get_image()[:, :, ::-1]
    instances2 = outputs2["instances"].to("cpu")
    num_objects2 = len(instances2)

    cv2.imshow("Homeplate Camera",output_frame2)
# ------------------------------------------------

print("Ball Coordinates:")
for point in ball_coordinates:
    print(point)

strike1 = False  # Initialize strike variable 선수 맞은편 카메라 스트라이크 판정 
strike2 = False # 홈플레이트 판정
final_strike = False # 최종 스트라이크 판정

for i in range(1, len(ball_coordinates)):
    ball_trajectory = (*ball_coordinates[i-1], *ball_coordinates[i])  # Get the line of ball's trajectory

    if drawn_line is not None and lines_intersect(ball_trajectory, drawn_line):
        strike1 = True
        break  # Exit the loop if a strike is detected
    else:
         strike1 = False

if num_objects2 > 0:
    strike2 = True
else:
    strike2 = False
        
print("측면 카메라:", strike1)
print("홈플레이트 카메라:", num_objects2)

# 스트라이크
if strike1 == True and strike2 == True:
    final_strike = True
    print("strike!!")
# 볼
else:
    final_strike = False
    print("ball") 
        
cap2.release()  #카메라 장치에서 받아온 메모리 해제
cv2.destroyAllWindows() #모든 윈도우 창 닫음

WARNING [06/16 21:26:22 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/16 21:26:22 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json
WARNING [06/16 21:26:22 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/16 21:26:22 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json


Some model parameters or buffers are not found in the checkpoint:
proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


Ball Coordinates:
측면 카메라: False
홈플레이트 카메라: 1
ball


In [2]:
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print(torch.cuda.is_available())

import detectron2
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
setup_logger()

import numpy as np
import os, json, cv2, random
import time

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import torchvision
import requests
print(torch.__version__, torch.cuda.is_available())

from detectron2.data.datasets import register_coco_instances
register_coco_instances("ball", {}, 'dataset/output_100.json', 'dataset')
register_coco_instances("bat", {}, 'dataset/output_100.json', 'dataset')

ball_metadata = MetadataCatalog.get("ball")
bat_metadata = MetadataCatalog.get("bat")

dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

from detectron2.engine import DefaultTrainer
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("ball", "bat")
cfg.DATASETS.TEST = ("ball", "bat")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")   # Let training initialize from model zoo
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # two class(ball, bat)

# ->> 처리
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_bat.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold for this model
cfg.DATASETS.TEST = ("ball", "bat")
predictor = DefaultPredictor(cfg)

#---- 선수 맞은편 카메라
cfg2 = get_cfg()
cfg2.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg2.DATASETS.TRAIN = ("ball",)
cfg2.DATASETS.TEST = ("ball", )
cfg2.DATALOADER.NUM_WORKERS = 2
cfg2.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")   # Let training initialize from model zoo
cfg2.MODEL.ROI_HEADS.NUM_CLASSES = 1 # one class(ball)

# ->> 처리
cfg2.MODEL.WEIGHTS = os.path.join(cfg2.OUTPUT_DIR, "model_final_ball.pth")
cfg2.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold for this model
cfg2.DATASETS.TEST = ("ball",)
predictor2 = DefaultPredictor(cfg2)
#----- 끝

# 함수 선언부
def lines_intersect(line1, line2):
    """Return True if line segment 'line1' and 'line2' intersect. False otherwise."""
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Compute vectors corresponding to lines
    px = x2 - x1
    py = y2 - y1
    qx = x4 - x3
    qy = y4 - y3

    # Cross product of vectors to determine if parallel
    cross_product = px * qy - qx * py
    if abs(cross_product) < 1e-8: # consider as zero
        return False  # parallel

    # Compute the intersection point
    dx = x1 - x3
    dy = y1 - y3
    t = (qx * dy - qy * dx) / cross_product
    u = (px * dy - py * dx) / cross_product

    return (0 <= t <= 1) and (0 <= u <= 1)

# 함수 선언부 끝

torch:  2.0 ; cuda:  2.0.0
detectron2: 0.2.1
True
2.0.0 True
WARNING [06/17 01:19:39 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/17 01:19:39 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json
WARNING [06/17 01:19:39 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/17 01:19:39 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json


Some model parameters or buffers are not found in the checkpoint:
proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [14]:
# 각 파일 path
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
 
# Create predictor and metadata objects
ball_metadata = MetadataCatalog.get("ball")
bat_metadata = MetadataCatalog.get("bat")
dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

predictor = DefaultPredictor(cfg)
predictor2 = DefaultPredictor(cfg2)

# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA) #벡엔드로 쿠다를 사용하여 속도향상을 꾀한다
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA) # 쿠다 디바이스에 계산 요청

inputWidth=320;
inputHeight=240;
inputScale=1.0/255;

start_time = time.time() # 시작 시간
ball_coordinates = []
points_buffer = {2: [], 8: [], 9: []} # 버퍼 초기화
drawn_line = None
line_timestamp = None  # 선의 타임스탬프 변수 초기화

# 선수 신체부위 추출용 끝

video_path1 = 'dataset/test19.mp4' # 측면카메라
cap1 = cv2.VideoCapture(video_path1)

video_path2 = 'dataset/test18.mp4' # 홈플레이트 카메라
cap2 = cv2.VideoCapture(video_path2)

while cv2.waitKey(1) <0:  #아무 키나 누르면 끝난다.
    #웹캠으로부터 영상 가져옴
    hasFrame, frame = cap1.read()  
    hasFrame2, frame2 = cap2.read() # 홈플레이트
    
    #웹캠으로부터 영상을 가져올 수 없으면 웹캠 중지
    if not hasFrame or not hasFrame2:
        cv2.waitKey()
        break
    
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    frameWidth2 = frame2.shape[1]
    frameHeight2 = frame2.shape[0]
    
    inpBlob = cv2.dnn.blobFromImage(frame, inputScale, (inputWidth, inputHeight), (0, 0, 0), swapRB=False, crop=False)
    
    imgb=cv2.dnn.imagesFromBlob(inpBlob)
    
    # network에 넣어주기
    net.setInput(inpBlob)

    # 결과 받아오기
    output = net.forward()
    
    outputs = predictor2(frame)
    v = Visualizer(frame[:, :, ::-1], metadata=ball_metadata)
    instances = outputs["instances"].to("cpu")
    confidence_threshold = 0.8 # Set your desired confidence threshold here

    for i in range(len(instances)):
        if instances.pred_classes[i] == 0: # 0번 클래스는 공입니다.
            confidence = instances.scores[i]
            if confidence > confidence_threshold:
                bbox = instances.pred_boxes[i]
                x1, y1, x2, y2 = bbox.tensor.numpy()[0]
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                radius = int((x2 - x1 + y2 - y1) / 4)
                cv2.circle(frame, (center_x, center_y), radius, (255, 0, 0), thickness=2)
                ball_coordinates.append((center_x, center_y))  # Add ball coordinates

    out = v.draw_instance_predictions(instances)

    # 키포인트 검출시 이미지에 그려줌
    points = []
    for i in [2,8,9]:
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]
    
        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (frameWidth * point[0]) / output.shape[3]
        y = (frameHeight * point[1]) / output.shape[2]

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
        if prob > 0.1 :   
            cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED) # circle(그릴곳, 원의 중심, 반지름, 색)
            cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
            points_buffer[i].append((x, y))  # 버퍼에 점 추가
        else :
            points.append(None)
            points_buffer[i] = []  # 버퍼 클리어

    # 각 점의 버퍼가 1초 이상의 데이터를 가지고 있다면
    if all([len(points_buffer[i]) >= 1*10 for i in [2, 8, 9]]):
        # 각 점의 평균 계산
        avg_points = {i: np.mean(points_buffer[i], axis=0) for i in [2, 8, 9]}
        shoulder_avg, waist_avg, knee_avg = avg_points[2], avg_points[8], avg_points[9]
        cal_avg = waist_avg[1] + ((shoulder_avg[1]-waist_avg[1])/2)

        # Draw the line
        drawn_line = (int(knee_avg[0]), int(knee_avg[1]), int(knee_avg[0]), int(cal_avg))
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)

    # 키 포인트 중 하나라도 감지되지 않으면, 최소 1초가 지났는지 확인
    if any(pt is None for pt in points):
        if line_timestamp is None or time.time() - line_timestamp >= 1:
            drawn_line = None
            line_timestamp = None

    # 이전에 선이 그려졌다면, 그 선을 그리고 타임스탬프를 업데이트
    if drawn_line is not None:
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)
        line_timestamp = time.time()
    
    cv2.imshow("Side Camera",frame)
# -----------홈플레이트 카메라 부분---------------
    outputs2 = predictor(frame2)
    v2 = Visualizer(frame2[:, :, ::-1], metadata=ball_metadata, scale=1)
    out2 = v2.draw_instance_predictions(outputs2["instances"].to("cpu"))
    output_frame2 = out2.get_image()[:, :, ::-1]
    output_frame2 = np.array(output_frame2)
    instances2 = outputs2["instances"].to("cpu")
    num_objects2 = len(instances2)

    strike_line_color = (0, 0, 255)  # 라인 색상 (빨간색)
    strike_line_left = int(frameWidth2 * (20 / 100))  # 좌측 라인 좌표
    strike_line_right = int(frameWidth2 * (80 / 100))  # 우측 라인 좌표
    
    center_line = frameHeight2 // 2

    cv2.line(output_frame2, (strike_line_left, 0), (strike_line_left, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (strike_line_right, 0), (strike_line_right, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (0, int(center_line)), (frameWidth2, int(center_line)), (0, 255, 0), 2)

    cv2.imshow("Homeplate Camera", output_frame2)

# ------------------------------------------------

print("Ball Coordinates:")
for point in ball_coordinates:
    print(point)

strike1 = False  # Initialize strike variable 선수 맞은편 카메라 스트라이크 판정 
strike2 = False # 홈플레이트 판정
final_strike = False # 최종 스트라이크 판정

for i in range(1, len(ball_coordinates)):
    ball_trajectory = (*ball_coordinates[i-1], *ball_coordinates[i])  # Get the line of ball's trajectory

    if drawn_line is not None and lines_intersect(ball_trajectory, drawn_line):
        strike1 = True
        break  # Exit the loop if a strike is detected
    else:
         strike1 = False

#---------------------홈플레이트 스트라이크/존 판정
# 좌우 스트라이크 존 사이를 지나가는 공이 있을 경우
ball_indices = [i for i, cls in enumerate(instances2.pred_classes) if cls == 0]  # 공 객체의 인덱스 추출

if any((instances2.pred_boxes.tensor[i][0] > strike_line_left and instances2.pred_boxes.tensor[i][0] < strike_line_right) or
       (instances2.pred_boxes.tensor[i][2] > strike_line_left and instances2.pred_boxes.tensor[i][2] < strike_line_right)
    for i in ball_indices): # 0:왼쪽상단 2:아래하단 x좌표
    strike2 = True
else:
    strike2 = False

#------------- 배트가 중앙 선을 넘어갈 경우 strike판정, strike2가 final strike가 되어야 함
bat_indices = [j for j, cls in enumerate(instances2.pred_classes) if cls == 1]  # 배트 객체의 인덱스 추출

if any(instances2.pred_boxes.tensor[j][1] < center_line or instances2.pred_boxes.tensor[j][3] < center_line
       for j in bat_indices): # 1:왼쪽상단 3:아래하단 y좌표
    strike2 = True
    final_strike = True
    print("배트가 중앙선을 넘었습니다.")
else:
    final_strike = False
#--------------------------
print("측면 카메라:", strike1)
print("strike_line_left:", strike_line_left)
print("strike_line_right", strike_line_right)
print("ball_indices", ball_indices)
print("홈플레이트 카메라:", num_objects2)
print("홈플레이트 카메라:", strike2)

# 스트라이크
if strike1 == True and strike2 == True or final_strike == True:
    final_strike = True
    print("strike!!")
# 볼
else:
    final_strike = False
    print("ball") 
        
cap2.release()  #카메라 장치에서 받아온 메모리 해제
cv2.destroyAllWindows() #모든 윈도우 창 닫음

WARNING [06/16 23:00:20 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/16 23:00:20 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json
WARNING [06/16 23:00:20 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/16 23:00:20 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json


Some model parameters or buffers are not found in the checkpoint:
proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


Ball Coordinates:
(379, 50)
(379, 50)
(371, 216)
(363, 411)
(356, 649)
측면 카메라: False
strike_line_left: 144
strike_line_right 576
ball_indices []
홈플레이트 카메라: 0
홈플레이트 카메라: False
ball


In [31]:
# 각 파일 path
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
 
# Create predictor and metadata objects
ball_metadata = MetadataCatalog.get("ball")
bat_metadata = MetadataCatalog.get("bat")
dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

predictor = DefaultPredictor(cfg)
predictor2 = DefaultPredictor(cfg2)

# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA) #벡엔드로 쿠다를 사용하여 속도향상을 꾀한다
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA) # 쿠다 디바이스에 계산 요청

inputWidth=320;
inputHeight=240;
inputScale=1.0/255;

start_time = time.time() # 시작 시간
ball_coordinates = []
ball_coordinates2 = []
points_buffer = {2: [], 8: [], 9: []} # 버퍼 초기화
drawn_line = None
line_timestamp = None  # 선의 타임스탬프 변수 초기화

# 선수 신체부위 추출용 끝

video_path1 = 'dataset/test24.mp4' # 측면카메라
cap1 = cv2.VideoCapture(video_path1)

video_path2 = 'dataset/test25.mp4' # 홈플레이트 카메라
cap2 = cv2.VideoCapture(video_path2)

while cv2.waitKey(1) <0:  #아무 키나 누르면 끝난다.
    #웹캠으로부터 영상 가져옴
    hasFrame, frame = cap1.read()  
    hasFrame2, frame2 = cap2.read() # 홈플레이트
    
    #웹캠으로부터 영상을 가져올 수 없으면 웹캠 중지
    if not hasFrame or not hasFrame2:
        cv2.waitKey()
        break
    
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    frameWidth2 = frame2.shape[1]
    frameHeight2 = frame2.shape[0]
    
    inpBlob = cv2.dnn.blobFromImage(frame, inputScale, (inputWidth, inputHeight), (0, 0, 0), swapRB=False, crop=False)
    
    imgb=cv2.dnn.imagesFromBlob(inpBlob)
    
    # network에 넣어주기
    net.setInput(inpBlob)

    # 결과 받아오기
    output = net.forward()
    
    outputs = predictor2(frame)
    v = Visualizer(frame[:, :, ::-1], metadata=ball_metadata)
    instances = outputs["instances"].to("cpu")
    confidence_threshold = 0.8 # Set your desired confidence threshold here

    for i in range(len(instances)):
        if instances.pred_classes[i] == 0: # 0번 클래스는 공입니다.
            confidence = instances.scores[i]
            if confidence > confidence_threshold:
                bbox = instances.pred_boxes[i]
                x1, y1, x2, y2 = bbox.tensor.numpy()[0]
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                radius = int((x2 - x1 + y2 - y1) / 4)
                cv2.circle(frame, (center_x, center_y), radius, (255, 0, 0), thickness=2)
                ball_coordinates.append((center_x, center_y))  # Add ball coordinates

    out = v.draw_instance_predictions(instances)

    # 키포인트 검출시 이미지에 그려줌
    points = []
    for i in [2,8,9]:
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]
    
        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (frameWidth * point[0]) / output.shape[3]
        y = (frameHeight * point[1]) / output.shape[2]

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
        if prob > 0.1 :   
            cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED) # circle(그릴곳, 원의 중심, 반지름, 색)
            cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
            points_buffer[i].append((x, y))  # 버퍼에 점 추가
        else :
            points.append(None)
            points_buffer[i] = []  # 버퍼 클리어

    # 각 점의 버퍼가 1초 이상의 데이터를 가지고 있다면
    if all([len(points_buffer[i]) >= 2*10 for i in [2, 8, 9]]):
        # 각 점의 평균 계산
        avg_points = {i: np.mean(points_buffer[i], axis=0) for i in [2, 8, 9]}
        shoulder_avg, waist_avg, knee_avg = avg_points[2], avg_points[8], avg_points[9]
        cal_avg = waist_avg[1] + ((shoulder_avg[1]-waist_avg[1])/2)

        # Draw the line
        drawn_line = (int(knee_avg[0]), int(knee_avg[1]), int(knee_avg[0]), int(cal_avg))
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)

    # 키 포인트 중 하나라도 감지되지 않으면, 최소 1초가 지났는지 확인
    if any(pt is None for pt in points):
        if line_timestamp is None or time.time() - line_timestamp >= 1:
            drawn_line = None
            line_timestamp = None

    # 이전에 선이 그려졌다면, 그 선을 그리고 타임스탬프를 업데이트
    if drawn_line is not None:
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)
        line_timestamp = time.time()
    
    cv2.imshow("Side Camera",frame)
# -----------홈플레이트 카메라 부분---------------
    outputs2 = predictor(frame2)
    v2 = Visualizer(frame2[:, :, ::-1], metadata=ball_metadata, scale=1)
    out2 = v2.draw_instance_predictions(outputs2["instances"].to("cpu"))
    output_frame2 = out2.get_image()[:, :, ::-1]
    output_frame2 = np.array(output_frame2)
    instances2 = outputs2["instances"].to("cpu")
    
    for i in range(len(instances2)):
        if instances2.pred_classes[i] == 0: # 0번 클래스는 공입니다.
                bbox = instances2.pred_boxes[i]
                x1, y1, x2, y2 = bbox.tensor.numpy()[0]
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                radius = int((x2 - x1 + y2 - y1) / 4)
                cv2.circle(frame, (center_x, center_y), radius, (255, 0, 0), thickness=2)
                ball_coordinates2.append((center_x, center_y))  # Add ball coordinates
                
    num_objects2 = len(instances2)

    strike_line_color = (0, 0, 255)  # 라인 색상 (빨간색)
    strike_line_left = int(frameWidth2 * (20 / 100))  # 좌측 라인 좌표
    strike_line_right = int(frameWidth2 * (80 / 100))  # 우측 라인 좌표
    
    center_line = frameHeight2 // 2

    cv2.line(output_frame2, (strike_line_left, 0), (strike_line_left, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (strike_line_right, 0), (strike_line_right, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (0, int(center_line)), (frameWidth2, int(center_line)), (0, 255, 0), 2)

    cv2.imshow("Homeplate Camera", output_frame2)

# ------------------------------------------------

# print("Ball Coordinates:")
# for point in ball_coordinates:
#     print(point)
    
# print("Ball Coordinates2:")
# for point in ball_coordinates2:
#     print(point)

strike1 = False  # Initialize strike variable 선수 맞은편 카메라 스트라이크 판정 
strike2 = False # 홈플레이트 판정
final_strike = False # 최종 스트라이크 판정
print("배트가 중앙선을 넘었습니다.")

#------------- 배트가 중앙 선을 넘어갈 경우 strike판정, strike2가 final strike가 되어야 함
bat_indices = [j for j, cls in enumerate(instances2.pred_classes) if cls == 1]  # 배트 객체의 인덱스 추출

if any(instances2.pred_boxes.tensor[j][1] < center_line or instances2.pred_boxes.tensor[j][3] < center_line
       for j in bat_indices): # 1:왼쪽상단 3:아래하단 y좌표
    final_strike = True
    print("배트가 중앙선을 넘었습니다.")
else:
    final_strike = True
    
#--------------------------

for i in range(1, len(ball_coordinates)):
    ball_trajectory = (*ball_coordinates[i-1], *ball_coordinates[i])  # Get the line of ball's trajectory

    if drawn_line is not None and lines_intersect(ball_trajectory, drawn_line):
        strike1 = True
        break  # Exit the loop if a strike is detected
    else:
         strike1 = False

#---------------------홈플레이트 스트라이크/존 판정
# 좌우 스트라이크 존 사이를 지나가는 공이 있을 경우
for ball_coord in ball_coordinates2:
    ball_x, ball_y = ball_coord # 공의 x, y 좌표 추출

    # 공의 x 좌표가 strike_line_left와 strike_line_right 사이에 위치하면 strike2를 True로 설정
    if strike_line_left <= ball_x <= strike_line_right:
        strike2 = True
    else:
        strike2 = False

#--------------------------
print("측면 카메라:", strike1)
# print("strike_line_left:", strike_line_left)
# print("strike_line_right", strike_line_right)
# print("ball_indices", ball_indices)
# print("홈플레이트 카메라:", num_objects2)
print("홈플레이트 카메라:", strike2)

# 스트라이크
if strike1 == True and strike2 == True or final_strike == True:
    final_strike = True
    print("strike!!")
# 볼
else:
    final_strike = False
    print("ball") 
        
cap2.release()  #카메라 장치에서 받아온 메모리 해제
cv2.destroyAllWindows() #모든 윈도우 창 닫음

WARNING [06/17 01:51:39 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/17 01:51:39 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json
WARNING [06/17 01:51:39 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/17 01:51:39 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json


Some model parameters or buffers are not found in the checkpoint:
proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


배트가 중앙선을 넘었습니다.
측면 카메라: False
홈플레이트 카메라: True
strike!!


In [26]:
# 각 파일 path
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
 
# Create predictor and metadata objects
ball_metadata = MetadataCatalog.get("ball")
bat_metadata = MetadataCatalog.get("bat")
dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

predictor = DefaultPredictor(cfg)
predictor2 = DefaultPredictor(cfg2)

# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA) #벡엔드로 쿠다를 사용하여 속도향상을 꾀한다
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA) # 쿠다 디바이스에 계산 요청

inputWidth=320;
inputHeight=240;
inputScale=1.0/255;

start_time = time.time() # 시작 시간
ball_coordinates = []
points_buffer = {2: [], 8: [], 9: []} # 버퍼 초기화
drawn_line = None
line_timestamp = None  # 선의 타임스탬프 변수 초기화

strike1 = False  # Initialize strike variable 선수 맞은편 카메라 스트라이크 판정 
strike2 = False # 홈플레이트 판정
final_strike = False # 최종 스트라이크 판정

# 선수 신체부위 추출용 끝

video_path1 = 'dataset/test19.mp4' # 측면카메라
cap1 = cv2.VideoCapture(video_path1)

video_path2 = 'dataset/test19.mp4' # 홈플레이트 카메라
cap2 = cv2.VideoCapture(video_path2)

while cv2.waitKey(1) <0:  #아무 키나 누르면 끝난다.
    #웹캠으로부터 영상 가져옴
    hasFrame, frame = cap1.read()  
    hasFrame2, frame2 = cap2.read() # 홈플레이트
    
    #웹캠으로부터 영상을 가져올 수 없으면 웹캠 중지
    if not hasFrame or not hasFrame2:
        cv2.waitKey()
        break
    
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    frameWidth2 = frame2.shape[1]
    frameHeight2 = frame2.shape[0]
    
    inpBlob = cv2.dnn.blobFromImage(frame, inputScale, (inputWidth, inputHeight), (0, 0, 0), swapRB=False, crop=False)
    
    imgb=cv2.dnn.imagesFromBlob(inpBlob)
    
    # network에 넣어주기
    net.setInput(inpBlob)

    # 결과 받아오기
    output = net.forward()
    
    outputs = predictor2(frame)
    v = Visualizer(frame[:, :, ::-1], metadata=ball_metadata)
    instances = outputs["instances"].to("cpu")
    confidence_threshold = 0.8 # Set your desired confidence threshold here

    for i in range(len(instances)):
        if instances.pred_classes[i] == 0: # 0번 클래스는 공입니다.
            confidence = instances.scores[i]
            if confidence > confidence_threshold:
                bbox = instances.pred_boxes[i]
                x1, y1, x2, y2 = bbox.tensor.numpy()[0]
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                radius = int((x2 - x1 + y2 - y1) / 4)
                cv2.circle(frame, (center_x, center_y), radius, (255, 0, 0), thickness=2)
                ball_coordinates.append((center_x, center_y))  # Add ball coordinates

    out = v.draw_instance_predictions(instances)

    # 키포인트 검출시 이미지에 그려줌
    points = []
    for i in [2,8,9]:
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]
    
        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (frameWidth * point[0]) / output.shape[3]
        y = (frameHeight * point[1]) / output.shape[2]

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
        if prob > 0.1 :   
            cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED) # circle(그릴곳, 원의 중심, 반지름, 색)
            cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
            points_buffer[i].append((x, y))  # 버퍼에 점 추가
        else :
            points.append(None)
            points_buffer[i] = []  # 버퍼 클리어

    # 각 점의 버퍼가 1초 이상의 데이터를 가지고 있다면
    if all([len(points_buffer[i]) >= 1*10 for i in [2, 8, 9]]):
        # 각 점의 평균 계산
        avg_points = {i: np.mean(points_buffer[i], axis=0) for i in [2, 8, 9]}
        shoulder_avg, waist_avg, knee_avg = avg_points[2], avg_points[8], avg_points[9]
        cal_avg = waist_avg[1] + ((shoulder_avg[1]-waist_avg[1])/2)

        # Draw the line
        drawn_line = (int(knee_avg[0]), int(knee_avg[1]), int(knee_avg[0]), int(cal_avg))
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)

    # 키 포인트 중 하나라도 감지되지 않으면, 최소 1초가 지났는지 확인
    if any(pt is None for pt in points):
        if line_timestamp is None or time.time() - line_timestamp >= 1:
            drawn_line = None
            line_timestamp = None

    # 이전에 선이 그려졌다면, 그 선을 그리고 타임스탬프를 업데이트
    if drawn_line is not None:
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)
        line_timestamp = time.time()
    
    cv2.imshow("Side Camera",frame)
# -----------홈플레이트 카메라 부분---------------
    outputs2 = predictor(frame2)
    v2 = Visualizer(frame2[:, :, ::-1], metadata=ball_metadata, scale=1)
    out2 = v2.draw_instance_predictions(outputs2["instances"].to("cpu"))
    output_frame2 = out2.get_image()[:, :, ::-1]
    output_frame2 = np.array(output_frame2)
    instances2 = outputs2["instances"].to("cpu")
    num_objects2 = len(instances2)

    strike_line_color = (0, 0, 255)  # 라인 색상 (빨간색)
    strike_line_left = frameWidth2 * 2 // 10  # 좌측 라인 좌표
    strike_line_right = frameWidth2 * 8 // 10  # 우측 라인 좌표
    center_line = frameHeight2 // 2

    cv2.line(output_frame2, (strike_line_left, 0), (strike_line_left, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (strike_line_right, 0), (strike_line_right, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (0, int(center_line)), (frameWidth2, int(center_line)), (0, 255, 0), 2)

    cv2.imshow("Homeplate Camera", output_frame2)

    ball_indices = [i for i, cls in enumerate(instances2.pred_classes) if cls == 0]  # 공 객체의 인덱스 추출

    if any((instances2.pred_boxes.tensor[i][0] > strike_line_left and instances2.pred_boxes.tensor[i][0] < strike_line_right) or
           (instances2.pred_boxes.tensor[i][2] > strike_line_left and instances2.pred_boxes.tensor[i][2] < strike_line_right)
           for i in ball_indices): # 0:왼쪽상단 2:아래하단 x좌표
        strike2 = True
    else:
        strike2 = False

    # 배트 객체 인식 확인
    bat_indices = [j for j, cls in enumerate(instances2.pred_classes) if cls == 1]  # 배트 객체의 인덱스 추출

    if any(instances2.pred_boxes.tensor[j][1] < center_line or instances2.pred_boxes.tensor[j][3] < center_line
           for j in bat_indices): # 1:왼쪽상단 3:아래하단 y좌표
        final_strike = True
    else:
        final_strike = False
# ------------------------------------------------

print("Ball Coordinates:")
for point in ball_coordinates:
    print(point)

for i in range(1, len(ball_coordinates)):
    ball_trajectory = (*ball_coordinates[i-1], *ball_coordinates[i])  # Get the line of ball's trajectory

    if drawn_line is not None and lines_intersect(ball_trajectory, drawn_line):
        strike1 = True
        break  # Exit the loop if a strike is detected
    else:
         strike1 = False

#--------------------------
print("측면 카메라 스트라이크 판정:", strike1)
print("홈플레이트 카메라 스트라이크 판정 : ", strike2)
print("최종 스트라이크 판별 : ", final_strike)

# 스트라이크
if strike1 == True and strike2 == True:
    final_strike = True
    print("양쪽 다 strike!!")
# 볼
# else:
#     final_strike = False
#     print("ball") 
        
cap2.release()  #카메라 장치에서 받아온 메모리 해제
cv2.destroyAllWindows() #모든 윈도우 창 닫음

# 2023-06-16

WARNING [06/16 23:31:56 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/16 23:31:56 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json
WARNING [06/16 23:31:56 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/16 23:31:56 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json


Some model parameters or buffers are not found in the checkpoint:
proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


Ball Coordinates:
(379, 50)
(379, 50)
(371, 216)
(363, 411)
측면 카메라 스트라이크 판정: False
홈플레이트 카메라 스트라이크 판정 :  True
최종 스트라이크 판별 :  False


# 11/2

In [1]:
# import torch, detectron2
# TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
# CUDA_VERSION = torch.__version__.split("+")[-1]
# print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# print("detectron2:", detectron2.__version__)
# print(torch.cuda.is_available())

# import detectron2
# from detectron2.utils.logger import setup_logger
# from detectron2.utils.visualizer import Visualizer
# setup_logger()

# import numpy as np
# import os, json, cv2, random

# from detectron2 import model_zoo
# from detectron2.engine import DefaultPredictor
# from detectron2.config import get_cfg
# from detectron2.utils.visualizer import Visualizer
# from detectron2.data import MetadataCatalog, DatasetCatalog

# import torch, torchvision
# print(torch.__version__, torch.cuda.is_available())

# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("ball", {}, 'dataset/output_100.json', 'dataset')
# register_coco_instances("bat", {}, 'dataset/output_100.json', 'dataset')

# ball_metadata = MetadataCatalog.get("ball")
# bat_metadata = MetadataCatalog.get("bat")

# dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

# from detectron2.engine import DefaultTrainer
# from detectron2.checkpoint import DetectionCheckpointer
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# from detectron2.data import MetadataCatalog, DatasetCatalog

# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# cfg.DATASETS.TRAIN = ("ball", "bat")
# cfg.DATASETS.TEST = ("ball", "bat")
# cfg.DATALOADER.NUM_WORKERS = 0
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")   # Let training initialize from model zoo
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# cfg.SOLVER.MAX_ITER = 2000   # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # two class (ball, bat)
# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
# trainer.resume_or_load(resume=False)
# trainer.train()

# # # 모델 저장
# trainer.checkpointer.save("model_final_output100")

import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print(torch.cuda.is_available())

import detectron2
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
setup_logger()

import numpy as np
import os, json, cv2, random
import time

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import torchvision
import requests
print(torch.__version__, torch.cuda.is_available())

from detectron2.data.datasets import register_coco_instances
register_coco_instances("ball", {}, 'dataset/output_100.json', 'dataset')
register_coco_instances("bat", {}, 'dataset/output_100.json', 'dataset')

ball_metadata = MetadataCatalog.get("ball")
bat_metadata = MetadataCatalog.get("bat")

dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

from detectron2.engine import DefaultTrainer
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("ball", "bat")
cfg.DATASETS.TEST = ("ball", "bat")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")   # Let training initialize from model zoo
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # two class(ball, bat)


# ->> 처리
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_output100.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold for this model
cfg.DATASETS.TEST = ("ball", "bat")
predictor = DefaultPredictor(cfg)

#---- 선수 맞은편 카메라
cfg2 = get_cfg()
cfg2.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg2.DATASETS.TRAIN = ("ball",)
cfg2.DATASETS.TEST = ("ball", )
cfg2.DATALOADER.NUM_WORKERS = 2
cfg2.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")   # Let training initialize from model zoo
cfg2.MODEL.ROI_HEADS.NUM_CLASSES = 1 # one class(ball)

# ->> 처리
cfg2.MODEL.WEIGHTS = os.path.join(cfg2.OUTPUT_DIR, "model_final.pth")
cfg2.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold for this model
cfg2.DATASETS.TEST = ("ball",)
predictor2 = DefaultPredictor(cfg2)
#----- 끝


torch:  2.0 ; cuda:  2.0.0
detectron2: 0.2.1
True
2.0.0 True
WARNING [11/02 16:17:21 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/02 16:17:21 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json
WARNING [11/02 16:17:21 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/02 16:17:21 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json


Some model parameters or buffers are not found in the checkpoint:
proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [32]:
# 함수 선언부
def lines_intersect(line1, line2):
    """Return True if line segment 'line1' and 'line2' intersect. False otherwise."""
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Compute vectors corresponding to lines
    px = x2 - x1
    py = y2 - y1
    qx = x4 - x3
    qy = y4 - y3

    # Cross product of vectors to determine if parallel
    cross_product = px * qy - qx * py
    if abs(cross_product) < 1e-8: # consider as zero
        return False  # parallel

    # Compute the intersection point
    dx = x1 - x3
    dy = y1 - y3
    t = (qx * dy - qy * dx) / cross_product
    u = (px * dy - py * dx) / cross_product

    return (0 <= t <= 1) and (0 <= u <= 1)

# 함수 선언부 끝

# 각 파일 path
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
 
# Create predictor and metadata objects
ball_metadata = MetadataCatalog.get("ball")
bat_metadata = MetadataCatalog.get("bat")
dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

predictor = DefaultPredictor(cfg)
predictor2 = DefaultPredictor(cfg2)

# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA) #벡엔드로 쿠다를 사용하여 속도향상을 꾀한다
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA) # 쿠다 디바이스에 계산 요청

inputWidth=320;
inputHeight=240;
inputScale=1.0/255;

start_time = time.time() # 시작 시간
ball_coordinates = []
points_buffer = {2: [], 8: [], 9: []} # 버퍼 초기화
drawn_line = None
line_timestamp = None  # 선의 타임스탬프 변수 초기화

strike1 = False  # Initialize strike variable 선수 맞은편 카메라 스트라이크 판정 
strike2 = False # 홈플레이트 판정
final_strike = False # 최종 스트라이크 판정

# 선수 신체부위 추출용 끝

video_path1 = '선수 맞은편 카메라.mp4' # 측면카메라
cap1 = cv2.VideoCapture(0)

# video_path2 = '공 중간으로 지나가는거.mp4' # 홈플레이트 카메라
video_path2 = '천장 2.mp4' # 홈플레이트 카메라

cap2 = cv2.VideoCapture(video_path2)

while cv2.waitKey(1) <0:  #아무 키나 누르면 끝난다.
    #웹캠으로부터 영상 가져옴
    hasFrame, frame = cap1.read()  
    hasFrame2, frame2 = cap2.read() # 홈플레이트
    
    #웹캠으로부터 영상을 가져올 수 없으면 웹캠 중지
    if not hasFrame or not hasFrame2:
        cv2.waitKey()
        break
        
#     # 프레임 수 계산
#     img = cv2.flip(frame, 1)
#     curTime = time.time()	# current time
#     fps = 1 / (curTime - start_time)
#     start_time = curTime
    
#     # 프레임 수 문자열에 저장
#     fps_str = "FPS : %0.1f" %fps

#     # 문자열 표시
#     cv2.putText(frame, fps_str, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0))
    
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    frameWidth2 = frame2.shape[1]
    frameHeight2 = frame2.shape[0]
    
    inpBlob = cv2.dnn.blobFromImage(frame, inputScale, (inputWidth, inputHeight), (0, 0, 0), swapRB=False, crop=False)
    
    imgb=cv2.dnn.imagesFromBlob(inpBlob)
    
    # network에 넣어주기
    net.setInput(inpBlob)

    # 결과 받아오기
    output = net.forward()
    
    outputs = predictor2(frame)
    v = Visualizer(frame[:, :, ::-1], metadata=ball_metadata)
    instances = outputs["instances"].to("cpu")
    confidence_threshold = 0.8 # Set your desired confidence threshold here

    for i in range(len(instances)):
        if instances.pred_classes[i] == 0: # 0번 클래스는 공입니다.
            confidence = instances.scores[i]
            if confidence > confidence_threshold:
                bbox = instances.pred_boxes[i]
                x1, y1, x2, y2 = bbox.tensor.numpy()[0]
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                radius = int((x2 - x1 + y2 - y1) / 4)
                cv2.circle(frame, (center_x, center_y), radius, (255, 0, 0), thickness=2)
                ball_coordinates.append((center_x, center_y))  # Add ball coordinates

    out = v.draw_instance_predictions(instances)

    # 키포인트 검출시 이미지에 그려줌
    points = []
    for i in [2,8,9]:
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]
    
        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (frameWidth * point[0]) / output.shape[3]
        y = (frameHeight * point[1]) / output.shape[2]

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
        if prob > 0.1 :   
            cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED) # circle(그릴곳, 원의 중심, 반지름, 색)
            cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
            points_buffer[i].append((x, y))  # 버퍼에 점 추가
        else :
            points.append(None)
            points_buffer[i] = []  # 버퍼 클리어

    # 각 점의 버퍼가 1초 이상의 데이터를 가지고 있다면
    if all([len(points_buffer[i]) >= 2*5 for i in [2, 8, 9]]):
        # 각 점의 평균 계산
        avg_points = {i: np.mean(points_buffer[i], axis=0) for i in [2, 8, 9]}
        shoulder_avg, waist_avg, knee_avg = avg_points[2], avg_points[8], avg_points[9]
        cal_avg = waist_avg[1] + ((shoulder_avg[1]-waist_avg[1])/2)

        # Draw the line
        drawn_line = (int(knee_avg[0]), int(knee_avg[1]), int(knee_avg[0]), int(cal_avg))
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)

    # 키 포인트 중 하나라도 감지되지 않으면, 최소 1초가 지났는지 확인
    if any(pt is None for pt in points):
        if line_timestamp is None or time.time() - line_timestamp >= 1:
            drawn_line = None
            line_timestamp = None

    # 이전에 선이 그려졌다면, 그 선을 그리고 타임스탬프를 업데이트
    if drawn_line is not None:
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)
        line_timestamp = time.time()
    
    cv2.imshow("Side Camera",frame)

    # ------------------------------------------------ 야구
    if strike1 == False: # 만약 측면 카메라 판정이 False일 때에만
        for i in range(1, len(ball_coordinates)):
            ball_trajectory = (*ball_coordinates[i-1], *ball_coordinates[i])  # Get the line of ball's trajectory

            if drawn_line is not None and lines_intersect(ball_trajectory, drawn_line):
                strike1 = True
            else:
                strike1 = False

  #  print("측면 카메라 판정 : ", strike1)
# -----------홈플레이트 카메라 부분---------------
    outputs2 = predictor(frame2)
    v2 = Visualizer(frame2[:, :, ::-1], metadata=ball_metadata, scale=1)
    out2 = v2.draw_instance_predictions(outputs2["instances"].to("cpu"))
    output_frame2 = out2.get_image()[:, :, ::-1]
    output_frame2 = np.array(output_frame2)
    instances2 = outputs2["instances"].to("cpu")
    num_objects2 = len(instances2)

    strike_line_color = (0, 0, 255)  # 라인 색상 (빨간색)
    strike_line_left = frameWidth2 * 2 // 10  # 좌측 라인 좌표
    strike_line_right = frameWidth2 * 8 // 10  # 우측 라인 좌표
    center_line = frameHeight2 // 2

    cv2.line(output_frame2, (strike_line_left, 0), (strike_line_left, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (strike_line_right, 0), (strike_line_right, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (0, int(center_line)), (frameWidth2, int(center_line)), (0, 255, 0), 2)

    cv2.imshow("Homeplate Camera", output_frame2)
    
    homeplate_confidence_threshold = 0.8 # Set your desired confidence threshold here

    ball_indices = [i for i, cls in enumerate(instances2.pred_classes) if cls == 0]  # 공 객체의 인덱스 추출
    
    if strike2 == False: # 만약 홈플레이트 카메라 판정이 False일 때에만
        if any((instances2.pred_boxes.tensor[i][0] > strike_line_left and instances2.pred_boxes.tensor[i][0] < strike_line_right) or
            (instances2.pred_boxes.tensor[i][2] > strike_line_left and instances2.pred_boxes.tensor[i][2] < strike_line_right)
            for i in ball_indices): # 0:왼쪽상단 2:아래하단 x좌표
            strike2 = True
          #  print("홈플레이트 카메라 판정 : ",strike2)
        else:
            strike2 = False
         #   print("홈플레이트 카메라 판정 : ",strike2)

    # 배트 객체 인식 확인
    bat_indices = [j for j, cls in enumerate(instances2.pred_classes) if cls == 1]  # 배트 객체의 인덱스 추출

    if any(instances2.pred_boxes.tensor[j][1] < center_line or instances2.pred_boxes.tensor[j][3] < center_line
           for j in bat_indices): # 1:왼쪽상단 3:아래하단 y좌표
        final_strike = True
        #print("배트가 중앙선을 넘었습니다.")

    else:
        final_strike = False

    # 스트라이크
    #if (strike1 == True and strike2 == True) or final_strike == True:
     #   break

    
print("측면 카메라:", strike1)
print("홈플레이트 카메라:", strike2)


print("Ball Coordinates:")
for point in ball_coordinates:
    print(point)

for i in range(1, len(ball_coordinates)):
    ball_trajectory = (*ball_coordinates[i-1], *ball_coordinates[i])  # Get the line of ball's trajectory

    if drawn_line is not None and lines_intersect(ball_trajectory, drawn_line):
        strike1 = True
        break  # Exit the loop if a strike is detected

        
# 스트라이크
if strike1 == True and strike2 == True or final_strike == True:
    print("strike!!")
# 볼
else:
    print("ball") 
        
cap2.release()  #카메라 장치에서 받아온 메모리 해제
cv2.destroyAllWindows() #모든 윈도우 창 닫음

WARNING [11/02 16:52:29 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/02 16:52:29 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json
WARNING [11/02 16:52:29 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/02 16:52:29 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json


Some model parameters or buffers are not found in the checkpoint:
proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


측면 카메라: False
홈플레이트 카메라: True
Ball Coordinates:
strike!!


# 복사본 (배트 인식도 수정)

In [6]:
# 함수 선언부
def lines_intersect(line1, line2):
    """Return True if line segment 'line1' and 'line2' intersect. False otherwise."""
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Compute vectors corresponding to lines
    px = x2 - x1
    py = y2 - y1
    qx = x4 - x3
    qy = y4 - y3

    # Cross product of vectors to determine if parallel
    cross_product = px * qy - qx * py
    if abs(cross_product) < 1e-8: # consider as zero
        return False  # parallel

    # Compute the intersection point
    dx = x1 - x3
    dy = y1 - y3
    t = (qx * dy - qy * dx) / cross_product
    u = (px * dy - py * dx) / cross_product

    return (0 <= t <= 1) and (0 <= u <= 1)

# 함수 선언부 끝

# 각 파일 path
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
 
# Create predictor and metadata objects
ball_metadata = MetadataCatalog.get("ball")
bat_metadata = MetadataCatalog.get("bat")
dataset_dicts = DatasetCatalog.get("ball") + DatasetCatalog.get("bat")

predictor = DefaultPredictor(cfg)
predictor2 = DefaultPredictor(cfg2)

# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA) #벡엔드로 쿠다를 사용하여 속도향상을 꾀한다
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA) # 쿠다 디바이스에 계산 요청

inputWidth=320;
inputHeight=240;
inputScale=1.0/255;

start_time = time.time() # 시작 시간
ball_coordinates = []
points_buffer = {2: [], 8: [], 9: []} # 버퍼 초기화
drawn_line = None
line_timestamp = None  # 선의 타임스탬프 변수 초기화

strike1 = False  # Initialize strike variable 선수 맞은편 카메라 스트라이크 판정 
strike2 = False # 홈플레이트 판정
final_strike = False # 최종 스트라이크 판정

# 선수 신체부위 추출용 끝

video_path1 = '선수 맞은편 카메라.mp4' # 측면카메라
cap1 = cv2.VideoCapture(1)

# video_path2 = '공 중간으로 지나가는거.mp4' # 홈플레이트 카메라
video_path2 = 'test2.mp4' # 홈플레이트 카메라

cap2 = cv2.VideoCapture(0)

while cv2.waitKey(1) <0:  #아무 키나 누르면 끝난다.
    #웹캠으로부터 영상 가져옴
    hasFrame, frame = cap1.read()  
    hasFrame2, frame2 = cap2.read() # 홈플레이트
    
    #웹캠으로부터 영상을 가져올 수 없으면 웹캠 중지
    if not hasFrame or not hasFrame2:
        cv2.waitKey()
        break
        
    # 프레임 수 계산
    img = cv2.flip(frame, 1)
    curTime = time.time()	# current time
    fps = 1 / (curTime - start_time)
    start_time = curTime
    
    # 프레임 수 문자열에 저장
    fps_str = "FPS : %0.1f" %fps

    # 문자열 표시
    cv2.putText(frame, fps_str, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0))
    
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    frameWidth2 = frame2.shape[1]
    frameHeight2 = frame2.shape[0]
    
    inpBlob = cv2.dnn.blobFromImage(frame, inputScale, (inputWidth, inputHeight), (0, 0, 0), swapRB=False, crop=False)
    
    imgb=cv2.dnn.imagesFromBlob(inpBlob)
    
    # network에 넣어주기
    net.setInput(inpBlob)

    # 결과 받아오기
    output = net.forward()
    
    outputs = predictor2(frame)
    v = Visualizer(frame[:, :, ::-1], metadata=ball_metadata)
    instances = outputs["instances"].to("cpu")
    confidence_threshold = 0.8 # Set your desired confidence threshold here

    for i in range(len(instances)):
        if instances.pred_classes[i] == 0: # 0번 클래스는 공입니다.
            confidence = instances.scores[i]
            if confidence > confidence_threshold:
                bbox = instances.pred_boxes[i]
                x1, y1, x2, y2 = bbox.tensor.numpy()[0]
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                radius = int((x2 - x1 + y2 - y1) / 4)
                cv2.circle(frame, (center_x, center_y), radius, (255, 0, 0), thickness=2)
                ball_coordinates.append((center_x, center_y))  # Add ball coordinates

    out = v.draw_instance_predictions(instances)

    # 키포인트 검출시 이미지에 그려줌
    points = []
    for i in [2,8,9]:
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]
    
        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (frameWidth * point[0]) / output.shape[3]
        y = (frameHeight * point[1]) / output.shape[2]

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
        if prob > 0.1 :   
            cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED) # circle(그릴곳, 원의 중심, 반지름, 색)
            cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
            points_buffer[i].append((x, y))  # 버퍼에 점 추가
        else :
            points.append(None)
            points_buffer[i] = []  # 버퍼 클리어

    # 각 점의 버퍼가 1초 이상의 데이터를 가지고 있다면
    if all([len(points_buffer[i]) >= 2*10 for i in [2, 8, 9]]):
        # 각 점의 평균 계산
        avg_points = {i: np.mean(points_buffer[i], axis=0) for i in [2, 8, 9]}
        shoulder_avg, waist_avg, knee_avg = avg_points[2], avg_points[8], avg_points[9]
        cal_avg = waist_avg[1] + ((shoulder_avg[1]-waist_avg[1])/2)

        # Draw the line
        drawn_line = (int(knee_avg[0]), int(knee_avg[1]), int(knee_avg[0]), int(cal_avg))
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)

    # 키 포인트 중 하나라도 감지되지 않으면, 최소 1초가 지났는지 확인
    if any(pt is None for pt in points):
        if line_timestamp is None or time.time() - line_timestamp >= 1:
            drawn_line = None
            line_timestamp = None

    # 이전에 선이 그려졌다면, 그 선을 그리고 타임스탬프를 업데이트
    if drawn_line is not None:
        cv2.line(frame, (drawn_line[0], drawn_line[1]), (drawn_line[2], drawn_line[3]), (0, 0, 255), 5)
        line_timestamp = time.time()
    
    cv2.imshow("Side Camera",frame)

    # ------------------------------------------------ 야구
    if strike1 == False: # 만약 측면 카메라 판정이 False일 때에만
        for i in range(1, len(ball_coordinates)):
            ball_trajectory = (*ball_coordinates[i-1], *ball_coordinates[i])  # Get the line of ball's trajectory

            if drawn_line is not None and lines_intersect(ball_trajectory, drawn_line):
                strike1 = True
            else:
                strike1 = False

    print("측면 카메라 판정 : ", strike1)
# -----------홈플레이트 카메라 부분---------------
    outputs2 = predictor(frame2)
    v2 = Visualizer(frame2[:, :, ::-1], metadata=ball_metadata, scale=1)
    out2 = v2.draw_instance_predictions(outputs2["instances"].to("cpu"))
    output_frame2 = out2.get_image()[:, :, ::-1]
    output_frame2 = np.array(output_frame2)
    instances2 = outputs2["instances"].to("cpu")
    num_objects2 = len(instances2)
    high_confidence_indices = [i for i, score in enumerate(instances2.scores) if score > 0.9]

    strike_line_color = (0, 0, 255)  # 라인 색상 (빨간색)
    strike_line_left = frameWidth2 * 2 // 10  # 좌측 라인 좌표
    strike_line_right = frameWidth2 * 8 // 10  # 우측 라인 좌표
    center_line = frameHeight2 // 2

    cv2.line(output_frame2, (strike_line_left, 0), (strike_line_left, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (strike_line_right, 0), (strike_line_right, frameHeight2), strike_line_color, 2)
    cv2.line(output_frame2, (0, int(center_line)), (frameWidth2, int(center_line)), (0, 255, 0), 2)

    cv2.imshow("Homeplate Camera", output_frame2)
    
#     ball_indices = [i for i, cls in enumerate(instances2.pred_classes) if (cls == 0 and instances2.scores[i] > 0.9)]
    ball_indices = [i for i in high_confidence_indices if instances2.pred_classes[i] == 0]
    
    if strike2 == False: # 만약 홈플레이트 카메라 판정이 False일 때에만
        if any((instances2.pred_boxes.tensor[i][0] > strike_line_left and instances2.pred_boxes.tensor[i][0] < strike_line_right) or
            (instances2.pred_boxes.tensor[i][2] > strike_line_left and instances2.pred_boxes.tensor[i][2] < strike_line_right)
            for i in ball_indices): # 0:왼쪽상단 2:아래하단 x좌표
            strike2 = True
            print("홈플레이트 카메라 판정 : ",strike2) 
        else:
            strike2 = False
            print("홈플레이트 카메라 판정 : ",strike2)

    # 배트 객체 인식 확인
#     bat_indices = [j for j, cls in enumerate(instances2.pred_classes) if (cls == 1 and instances2.scores[j] > 0.9)]  # 배트 객체의 인덱스 추출
    bat_indices = [i for i in high_confidence_indices if instances2.pred_classes[i] == 1]
    
    if any(instances2.pred_boxes.tensor[j][1] < center_line or instances2.pred_boxes.tensor[j][3] < center_line
           for j in bat_indices): # 1:왼쪽상단 3:아래하단 y좌표
        final_strike = True
        print("배트가 중앙선을 넘었습니다.")

    else:
        final_strike = False

    # 스트라이크
    #if (strike1 == True and strike2 == True) or final_strike == True:
     #   break

print("측면 카메라:", strike1)
print("홈플레이트 카메라:", strike2)

# 스트라이크
if strike1 == True and strike2 == True or final_strike == True:
    print("strike!!")
# 볼
else:
    print("ball") 
        
cap2.release()  #카메라 장치에서 받아온 메모리 해제
cv2.destroyAllWindows() #모든 윈도우 창 닫음

WARNING [11/02 16:24:16 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/02 16:24:16 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json
WARNING [11/02 16:24:16 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/02 16:24:16 d2.data.datasets.coco]: Loaded 100 images in COCO format from dataset/output_100.json


Some model parameters or buffers are not found in the checkpoint:
proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


측면 카메라 판정 :  False
홈플레이트 카메라 판정 :  False
측면 카메라 판정 :  False
홈플레이트 카메라 판정 :  True
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False

측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라 판정 :  False
배트가 중앙선을 넘었습니다.
측면 카메라: False
홈플레이트 카메라: True
strike!!
